In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pathlib

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
data_dir = './VPN_Traffic/'

batch_size = 32
img_height = 32
img_width = 32

In [3]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels="inferred",
    validation_split=0.2,
    subset="training",
    seed=69,
    color_mode="grayscale",
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 190473 files belonging to 6 classes.
Using 152379 files for training.


In [4]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels="inferred",
    validation_split=0.2,
    subset="validation",
    seed=42,
    color_mode="grayscale",
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 190473 files belonging to 6 classes.
Using 38094 files for validation.


In [5]:
class_names = train_ds.class_names
print(class_names)

['audio', 'chat', 'email', 'files', 'sftp', 'video']


In [6]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 32, 32, 1)
(32,)


In [7]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)